# W3P2 - PART3 - yelp_foursquareEDA

assignment file part 2

In [1]:
from datetime import datetime
import pandas as pd
import requests
import json
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. Assuming the average comfortable walking speed below, the radius of 1km should list sites available within a 15min walking distance from each bike station.

$$
1km * hour/4km 
$$

In [2]:
# Import data from file
df = pd.read_csv('../data/df_ctybks_toronto.csv')
df.shape

(655, 19)

Create lat and long series from the ctyBks table:

In [3]:
lat = df['latitude']
long = df['longitude']
print(f'{lat[5]},{long[5]}')

43.657763,-79.389165


In [4]:
numStns = len(lat)

In [5]:
def fsqStationPOI(lat, long):
  '''
  input:
  output:
  '''
  api_key = os.environ["FOURSQUARE_API_KEY"]
  fields = 'fsq_id,name,geocodes,categories,distance,rating,hours,hours_popular,popularity,features'

  headers = {
      "accept": "application/json",
      "Authorization": api_key
  }

  fsq_url = (f'https://api.foursquare.com/v3/places/search?ll={lat},{long}&fields={fields}&radius=1000&&limit=50')
  
  fsqStnPOI_json = requests.get(fsq_url, headers=headers).json()

  list_of_dict = []
  for poi in fsqStnPOI_json['results']:
    categories = poi.get('categories', None)
    cat_id = categories[0]['id'] if len(categories) > 0 else None
    category_name = categories[0]['name'] if len(categories) > 0 else None

    poi_dict = {
            'lat-long': f'{lat},{long}'
          , 'fsq_id': poi['fsq_id']
          , 'cat_id': cat_id
          , 'category_name': category_name
          , 'categories': categories
          , 'name': poi.get('name', None)
          , 'distance': poi.get('distance', None)
          , 'latitude': poi.get('geocodes', {}).get('main', {}).get('latitude', None)
          , 'longitude': poi.get('geocodes', {}).get('main', {}).get('longitude', None)
          , 'address': poi.get('location', {}).get('formatted_address', None)
          , 'popularity': poi.get('popularity', None)
          , 'open_now': poi.get('hours', {}).get('open_now', None)
          , 'rating': poi.get('rating', None)
          , 'rating_count': poi.get('stats', {}).get('total_ratings', None)
          , 'my_timestamp': datetime.now()
      }
      
    list_of_dict.append(poi_dict)
  return pd.DataFrame(list_of_dict)


In [6]:
# datetime.astimezone(tz=pytz.timezone('America/Toronto'))

In [7]:
# I guess I'll figure out how to convert it later..

# utc = '2022-12-03T18:32:24.600000Z'
# date = datetime.strptime(utc, '%Y-%m-%dT%H:%M:%S.%fZ')
# converted = date.astimezone(tz=pytz.timezone('America/Toronto'))

# print(converted)

In [8]:
print(f'{lat[5]},{long[5]}')

43.657763,-79.389165


In [9]:
# Test
fsqStnPOI_df = fsqStationPOI(lat[5], long[5])
fsqStnPOI_df.head(1)

,lat-long,fsq_id,cat_id,category_name,categories,name,distance,latitude,longitude,address,popularity,open_now,rating,rating_count,my_timestamp
0,"43.657763,-79.389165",537d4d6d498ec171ba22e7fe,13034,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Jimmy's Coffee,294,43.658525,-79.385436,None,0.972089,False,8.5,None,2022-12-03 19:38:44.660853


Generate a list of DataFrames now for each station site:

In [10]:
fsq_dfs_list = []
for i in range(numStns):
  poi_df = fsqStationPOI(lat[i], long[i])
  fsq_dfs_list.append(poi_df)

Concatenate the DataFrames to create one large one.

In [11]:
df_fsq = x = pd.DataFrame()
x = x = pd.DataFrame()

# concatenate full df object of all stn site data:
for poi_df in fsq_dfs_list:
    x = pd.concat([df_fsq, poi_df])
    df_fsq = x

In [14]:
df_fsq.shape

(32709, 15)

In [13]:
df_fsq.to_csv(f'../data/fsqPOI/df_fsq{datetime.now()}.csv', index=False)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating